Admission campaign analysis


In [78]:
import pandas as pd
import re
from IPython.display import display

def extract_metadata_with_pandas(html_path):
    tables = pd.read_html(html_path, header=None)
    df = tables[0]  # first table with metadata + main data
    
    # Flatten to a single column series of strings
    all_texts = df.astype(str).stack()
    
    metadata = {}
    for text in all_texts:
        if " - " in text:
            key, value = text.split(" - ", 1)
            metadata[key.strip()] = value.strip()
    
    # Extract year
    date_text = metadata.get("Дата формирования", "")
    match_year = re.search(r"\d{4}", date_text)
    metadata["Год"] = int(match_year.group()) if match_year else None
    
    # Extract total places
    places_text = next((t for t in all_texts if "Всего мест" in t), "")
    match_places = re.search(r"Всего мест:\s*(\d+)", places_text)
    metadata["Количество мест"] = int(match_places.group(1)) if match_places else None
    
    return {
        "Подразделение": metadata.get("Подразделение"),
        "Уровень подготовки": metadata.get("Уровень подготовки"),
        "Направление подготовки/специальность": metadata.get("УГС/Направление подготовки/специальность"),
        "Год": metadata.get("Год"),
        "Количество мест": metadata.get("Количество мест")
    }


def extract_normalized_dataframe(html_path: str):
    table = pd.read_html(io=html_path, flavor="lxml")[0]
    table = table.map(lambda cell: cell.strip() if isinstance(cell, str) else cell)
    
    # Find the table content and extract it
    rows = table.values.tolist()
    header_row_index = None
    for i, row in enumerate(rows):
        if "Сумма баллов" in row and "Учебная группа" in row:
            header_row_index = i
            break
    
    if header_row_index is None:
        raise ValueError("Header row not found")
    
    data_rows = rows[header_row_index+1:]  
    headers = rows[header_row_index]      

    # Create a new DataFrame with proper headers
    data_frame = pd.DataFrame(data_rows, columns=headers)

    # Remove empty-value columns
    empty_cols = data_frame.columns[data_frame.isna().all()]
    data_frame.drop(columns=empty_cols, inplace=True)

    # Remove redundant columns
    cols_to_remove = ["№", "Уникальный код", "Представление приказа", "Учебная группа"]
    existing_cols_to_remove = [col for col in cols_to_remove if col in data_frame.columns]
    data_frame.drop(columns=existing_cols_to_remove, inplace=True)

    # print(f"{data_frame.dtypes}\n")
    data_frame = data_frame.apply(pd.to_numeric)
    # print(f"{data_frame.dtypes}\n")
    
    return data_frame

path_to_html = "https://pstu.ru/files/file/Abitur/2025%20final/%D0%93%D1%83%D0%BC%D0%A4_%D0%91_%D0%9D_000006996.html"
extracted_df = extract_normalized_dataframe(path_to_html)
summary_data = extract_metadata_with_pandas(path_to_html)
# display(extracted_df)

mean_total = extracted_df['Сумма баллов'].mean()
min_total  = extracted_df['Сумма баллов'].min()
max_total  = extracted_df['Сумма баллов'].max()
display(f"Mean: {mean_total}, Minimum: {min_total}, Maximum: {max_total}")

# Evaluate mean values for 3 subject types 
total_col_idx = extracted_df.columns.get_loc("Сумма баллов")
subject_cols = extracted_df.columns[total_col_idx + 1 : total_col_idx + 4]
subject_means = extracted_df[subject_cols].mean()
print("Average scores per subject:")
print(subject_means)

# Get max mean value within 3 subjects
best_subject = subject_means.idxmax()
print("Subject with highest average:", best_subject)


# Detect Russian column among subjects
rus_col = [col for col in subject_cols if "Русский" in col][0]
rus_mean = subject_means[rus_col]

other_subjects = [c for c in subject_cols if c != rus_col]
other_means = subject_means[other_subjects]

bool_mask_type1 = (extracted_df[rus_col] < rus_mean) & ((extracted_df[other_subjects] > other_means).all(axis=1))
type1_count = bool_mask_type1.sum()
print("Number of Тип_1 students:", type1_count)

bool_mask_type2 = (extracted_df[rus_col] > rus_mean) & ((extracted_df[other_subjects] < other_means).all(axis=1))
type2_count = bool_mask_type2.sum()
print("Number of Тип_2 students:", type2_count)

summary_data.update({
    "Предметы ЕГЭ": "; ".join(subject_cols),
    "Средняя сумма баллов": mean_total,
    "Min сумма баллов": min_total, 
    "Max сумма баллов": max_total,
    "Предмет с высшим средним": best_subject,
    "Кол-во студентов тип_1": type1_count,
    "Кол-во студентов тип_2": type2_count,
})

summary_df = pd.DataFrame([summary_data])
display(summary_df)


'Mean: 233.26666666666668, Minimum: 224, Maximum: 256'

Average scores per subject:
ЧиО / Обществознание                               75.200000
АиНМА / Иностранный язык / История / Математика    75.733333
Русский язык                                       82.333333
dtype: float64
Subject with highest average: Русский язык
Number of Тип_1 students: 2
Number of Тип_2 students: 3


,Подразделение,Уровень подготовки,Направление подготовки/специальность,Год,Количество мест,Предметы ЕГЭ,Средняя сумма баллов,Min сумма баллов,Max сумма баллов,Предмет с высшим средним,Кол-во студентов тип_1,Кол-во студентов тип_2
0,Гуманитарный факультет,Бакалавриат,Государственное и муниципальное управление,2025,15,ЧиО / Обществознание; АиНМА / Иностранный язык...,233.266667,224,256,Русский язык,2,3
